In [2]:
import pandas as pd
order=pd.read_csv("orders.csv")
print(order)

      order_id  user_id  restaurant_id  order_date  total_amount  \
0            1     2508            450  18-02-2023        842.97   
1            2     2693            309  18-01-2023        546.68   
2            3     2084            107  15-07-2023        163.93   
3            4      319            224  04-10-2023       1155.97   
4            5     1064            293  25-12-2023       1321.91   
...        ...      ...            ...         ...           ...   
9995      9996     2528            249  21-05-2023       1211.96   
9996      9997     2867            267  06-08-2023       1188.05   
9997      9998      522            420  11-11-2023        979.44   
9998      9999      319            492  08-09-2023       1105.93   
9999     10000      457            439  21-10-2023        879.58   

                     restaurant_name  
0                  New Foods Chinese  
1     Ruchi Curry House Multicuisine  
2              Spice Kitchen Punjabi  
3             Darbar Kitche

In [3]:
import pandas as pd
order=pd.read_csv("orders.csv")
print(order)

      order_id  user_id  restaurant_id  order_date  total_amount  \
0            1     2508            450  18-02-2023        842.97   
1            2     2693            309  18-01-2023        546.68   
2            3     2084            107  15-07-2023        163.93   
3            4      319            224  04-10-2023       1155.97   
4            5     1064            293  25-12-2023       1321.91   
...        ...      ...            ...         ...           ...   
9995      9996     2528            249  21-05-2023       1211.96   
9996      9997     2867            267  06-08-2023       1188.05   
9997      9998      522            420  11-11-2023        979.44   
9998      9999      319            492  08-09-2023       1105.93   
9999     10000      457            439  21-10-2023        879.58   

                     restaurant_name  
0                  New Foods Chinese  
1     Ruchi Curry House Multicuisine  
2              Spice Kitchen Punjabi  
3             Darbar Kitche

In [6]:
import pandas as pd
user=pd.read_json("users.json")
print(user)

      user_id       name       city membership
0           1     User_1    Chennai    Regular
1           2     User_2       Pune       Gold
2           3     User_3  Bangalore       Gold
3           4     User_4  Bangalore    Regular
4           5     User_5       Pune       Gold
...       ...        ...        ...        ...
2995     2996  User_2996  Hyderabad       Gold
2996     2997  User_2997  Hyderabad    Regular
2997     2998  User_2998  Bangalore    Regular
2998     2999  User_2999       Pune    Regular
2999     3000  User_3000  Hyderabad       Gold

[3000 rows x 4 columns]


In [25]:
import sqlite3
conn=sqlite3.connect("restaurants.db")
cursor=conn.cursor()
with open("restaurants.sql","r") as f:
    sql_script=f.read()

cursor.executescript(sql_script)
conn.commit()
conn.close()

In [26]:
conn=sqlite3.connect("restaurants.db")
cursor=conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())
conn.close()

[('restaurants',)]


In [27]:
import sqlite3
import pandas as pd
conn=sqlite3.connect("restaurants.db")
tables=pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';",conn)
print("Tables in DB:",tables)
restaurants=pd.read_sql_query("SELECT * FROM restaurants",conn)
print(restaurants.head())
conn.close()

Tables in DB:           name
0  restaurants
   restaurant_id restaurant_name  cuisine  rating
0              1    Restaurant_1  Chinese     4.8
1              2    Restaurant_2   Indian     4.1
2              3    Restaurant_3  Mexican     4.3
3              4    Restaurant_4  Chinese     4.1
4              5    Restaurant_5  Chinese     4.8


In [30]:
orders_users=pd.merge(order,user,on="user_id",how="left")

In [31]:
final_dataset=pd.merge(
    orders_users,
    restaurants,
    on="restaurant_id",
    how="left"
)

In [32]:
print(final_dataset.head())
print(final_dataset.info())

   order_id  user_id  restaurant_id  order_date  total_amount  \
0         1     2508            450  18-02-2023        842.97   
1         2     2693            309  18-01-2023        546.68   
2         3     2084            107  15-07-2023        163.93   
3         4      319            224  04-10-2023       1155.97   
4         5     1064            293  25-12-2023       1321.91   

                restaurant_name_x       name       city membership  \
0               New Foods Chinese  User_2508  Hyderabad    Regular   
1  Ruchi Curry House Multicuisine  User_2693       Pune    Regular   
2           Spice Kitchen Punjabi  User_2084    Chennai       Gold   
3          Darbar Kitchen Non-Veg   User_319  Bangalore       Gold   
4       Royal Eatery South Indian  User_1064       Pune    Regular   

  restaurant_name_y  cuisine  rating  
0    Restaurant_450  Mexican     3.2  
1    Restaurant_309   Indian     4.5  
2    Restaurant_107  Mexican     4.0  
3    Restaurant_224  Chinese    

In [33]:
final_dataset.to_csv("final_food_delivery_dataset.csv",index=False)

In [35]:
import pandas as pd
df=pd.read_csv("final_food_delivery_dataset.csv")
df["order_date"]=pd.to_datetime(df["order_date"],dayfirst=True)

In [37]:
order_trends=(df.groupby(df["order_date"].dt.to_period("M")).agg(total_orders=("order_id","count"),tota_revenue=("total_amount","sum")).reset_index())
print(order_trends)

   order_date  total_orders  tota_revenue
0     2023-01           804     645818.76
1     2023-02           785     630867.40
2     2023-03           903     716738.98
3     2023-04           812     666097.18
4     2023-05           844     668428.61
5     2023-06           784     610822.93
6     2023-07           859     688559.45
7     2023-08           851     694987.58
8     2023-09           812     653838.07
9     2023-10           863     699187.73
10    2023-11           807     637772.43
11    2023-12           849     681303.50
12    2024-01            27      17201.50


In [38]:
user_behavior=(df.groupby("user_id").agg(total_orders=("order_id","count"),
                                         total_spent=("total_amount","sum"),
                                         avg_order_value=("total_amount","mean")).reset_index())
print(user_behavior.head())

   user_id  total_orders  total_spent  avg_order_value
0        1             1      1289.66         1289.660
1        2            10      7564.12          756.412
2        3             2      1839.51          919.755
3        4             4      3741.16          935.290
4        5             5      5742.88         1148.576


In [40]:
city_performance=(df.groupby("city").agg(total_orders=("order_id","count"),
                  total_revenue=("total_amount","sum")).sort_values(by="total_revenue",ascending=False).reset_index())
print(city_performance)

        city  total_orders  total_revenue
0  Bangalore          2751     2206946.58
1    Chennai          2469     1990513.03
2       Pune          2430     1924797.93
3  Hyderabad          2350     1889366.58


In [41]:
cuisine_performance=(df.groupby("cuisine").agg(total_orders=("order_id","count"),
                  total_revenue=("total_amount","sum")).sort_values(by="total_revenue",ascending=False).reset_index())
print(cuisine_performance)

   cuisine  total_orders  total_revenue
0  Mexican          2581     2085503.09
1  Italian          2532     2024203.80
2   Indian          2469     1971412.58
3  Chinese          2418     1930504.65


In [49]:
membership_analysis=(df.groupby("membership").agg(total_orders=("order_id","count"),
                  total_revenue=("total_amount","sum"),avg_order_value=("total_amount","mean")).reset_index())
print(membership_analysis)

  membership  total_orders  total_revenue  avg_order_value
0       Gold          4987     3975364.89       797.145556
1    Regular          5013     4036259.23       805.158434


In [50]:
gold_revenue=df[df["membership"]=="Gold"]["total_amount"].sum()
total_revenue=df["total_amount"].sum()
gold_percentage=(gold_revenue/total_revenue)*100
print(f"Gold Members Contribution: {gold_percentage:.2f}%")

Gold Members Contribution: 49.62%


In [51]:
monthly_revenue=(df.groupby(df["order_date"].dt.month).agg(total_revenue=("total_amount","sum")).reset_index())
print(monthly_revenue)

    order_date  total_revenue
0            1      663020.26
1            2      630867.40
2            3      716738.98
3            4      666097.18
4            5      668428.61
5            6      610822.93
6            7      688559.45
7            8      694987.58
8            9      653838.07
9           10      699187.73
10          11      637772.43
11          12      681303.50


In [52]:
top_restaurants=(df.groupby("restaurant_name_x").agg(total_revenue=("total_amount","sum")).sort_values(by="total_revenue",ascending=False).head(10).reset_index())
print(top_restaurants)

                 restaurant_name_x  total_revenue
0              Spice Foods Non-Veg       48616.39
1   Annapurna Kitchen North Indian       46462.08
2        Swagath Cafe Multicuisine       45821.74
3   Hotel Curry House North Indian       45091.67
4        Amma Biryani North Indian       44169.33
5     Swagath Kitchen North Indian       42727.16
6  Darbar Eatery Family Restaurant       41742.61
7              Sai Biryani Non-Veg       40942.97
8           Amma Cafe South Indian       40352.80
9           Sai Restaurant Non-Veg       40190.74


In [56]:
import pandas as pd
df=pd.read_csv("final_food_delivery_dataset.csv")
gold_mem=df[df["membership"]=="Gold"]
gold_city_revenue=(gold_mem.groupby("city")["total_amount"].sum().reset_index().sort_values(by="total_amount", ascending=False))
print(gold_city_revenue.head(1))

      city  total_amount
1  Chennai    1080909.79


In [57]:
import pandas as pd
df=pd.read_csv("final_food_delivery_dataset.csv")
cuisine_aov=(df.groupby("cuisine")["total_amount"].mean().reset_index().sort_values(by="total_amount", ascending=False))
print(cuisine_aov.head(1))

   cuisine  total_amount
3  Mexican    808.021344


In [59]:
import pandas as pd
df=pd.read_csv("final_food_delivery_dataset.csv")
user_tas=(df.groupby("user_id")["total_amount"].sum().reset_index())
high_value_users=user_tas[user_tas["total_amount"]>1000]
count_users=high_value_users["user_id"].nunique()
print(count_users)

2544


In [61]:
import pandas as pd
df=pd.read_csv("final_food_delivery_dataset.csv")
bins=[0,2.9,3.5,4.0,4.5,5.0]
labels=["1.0-2.9","3.0-3.5","3.6-4.0","4.1-4.5","4.6-5.0"]
df["rating_range"]=pd.cut(df["rating"],bins=bins,labels=labels)
rating_revenue=(df.groupby("rating_range")["total_amount"].sum().reset_index().sort_values(by="total_amount", ascending=False))
print(rating_revenue.head(1))

  rating_range  total_amount
3      4.6-5.0    2197030.75


In [62]:
import pandas as pd
df=pd.read_csv("final_food_delivery_dataset.csv")
gold_mem=df[df["membership"]=="Gold"]
gold_city_aov=(gold_mem.groupby("city")["total_amount"].mean().reset_index().sort_values(by="total_amount", ascending=False))
print(gold_city_aov.head(1))

      city  total_amount
1  Chennai     808.45908


In [63]:
import pandas as pd
df=pd.read_csv("final_food_delivery_dataset.csv")
cuisine_analysis=(df.groupby("cuisine").agg(distinct_restaurants=("restaurant_id","nunique"),total_revenue=("total_amount","sum")).reset_index())
cuisine_insight=cuisine_analysis.sort_values(by=["distinct_restaurants","total_revenue"],ascending=[True,False])
print(cuisine_insight)

   cuisine  distinct_restaurants  total_revenue
0  Chinese                   120     1930504.65
2  Italian                   126     2024203.80
1   Indian                   126     1971412.58
3  Mexican                   128     2085503.09


In [67]:
avg_revenue=cuisine_analysis["total_revenue"].mean()
significant_cuisines=cuisine_analysis[cuisine_analysis["total_revenue"]>avg_revenue].sort_values(by="distinct_restaurants")
print(significant_cuisines.head(1))

   cuisine  distinct_restaurants  total_revenue
2  Italian                   126      2024203.8


In [69]:
import pandas as pd
df=pd.read_csv("final_food_delivery_dataset.csv")
total_orders=df["order_id"].nunique()
gold_orders=df[df["membership"]=="Gold"]["order_id"].nunique()
gold_percentage=round((gold_orders/total_orders)*100)
print(gold_percentage)

50


In [76]:
import pandas as pd
df=pd.read_csv("final_food_delivery_dataset.csv")
restaurant_stats=(df.groupby("restaurant_name_x").agg(total_orders=("order_id","nunique"),avg_order_value=("total_amount","mean")).reset_index())
low_value_restaurants=restaurant_stats[restaurant_stats["total_orders"]<20]
result=low_value_restaurants.sort_values(by="avg_order_value",ascending=False)
print(result.head(1))

            restaurant_name_x  total_orders  avg_order_value
173  Hotel Dhaba Multicuisine            13      1040.222308


In [78]:
import pandas as pd
df=pd.read_csv("final_food_delivery_dataset.csv")
combo_revenue=(df.groupby(["membership","cuisine"])["total_amount"].sum().reset_index().sort_values(by="total_amount", ascending=False))
print(combo_revenue.head(1))

  membership  cuisine  total_amount
7    Regular  Mexican     1072943.3
